In [1]:
import os
import json
import tracemalloc
import pandas as pd

In [2]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain , SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
key=os.getenv("OPENAI_API_KEY")

In [5]:
if key:
    print("Key loaded successfully")
else:
    print("Key not found")

Key loaded successfully


In [6]:
from langchain.chat_models import ChatOpenAI
llm=ChatOpenAI(openai_api_key=key,model_name='gpt-3.5-turbo', temperature=0.5)

C:\Users\HP\AppData\Local\Temp\ipykernel_6544\3923787719.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(openai_api_key=key,model_name='gpt-3.5-turbo', temperature=0.5)


In [8]:
RESPONSE_JSON={
    '1':{
        'mcq':'multiple choice question',
        'options':{
            'a':'choice here',
            'b':'choice here',
            'c':'choice here',
            'd':'choice here'
        },
        'correct option':'correct answer'
    },
    '2':{
        'mcq':'multiple choice question',
        'options':{
            'a':'choice here',
            'b':'choice here',
            'c':'choice here',
            'd':'choice here'
        },
        'correct option':'correct answer'
    },
    '3':{
        'mcq':'multiple choice question',
        'options':{
            'a':'choice here',
            'b':'choice here',
            'c':'choice here',
            'd':'choice here'
        },
        'correct option':'correct answer'
    }
}
 

In [9]:
TEMPLATE='''
        Text:{text}
        you are an expert in MCQ maker.Given the above text,it is your job to create a quiz of {number}/
        Multiple choice question for {subject} in tone {tone}.
        Make sure the questions are not repeated and check all questions to be conforming the text as well.
        Make sure to formate your response like RESPONSE_JSON and use it as guide.
        Ensure to make {number} of MCQs
        ### RESPONSE_JSON
        {response_json}

'''

In [10]:
quiz_generation_prompt=PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template=TEMPLATE
)

In [14]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_6544\2669661367.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [15]:
TEMPLATE2='''
            you are an expert english grammarian and writer.
            givean a multiple choice quiz for {subject} students.
            you need to evaluate the complexity of the question and given a complete analysis of the quiz. only use at max  word complexity,
            Update the quiz questions which needs to be changeed and chance the tone such that it prefectly fits students ability.
            Quiz_MCQs
            {quiz}
            check from an expert english writer of the above quiz
            '''

In [17]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=['subject','quiz'],
    template=TEMPLATE2
)

In [18]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key='review',verbose=True)

In [19]:
generats_evalution_chain=SequentialChain(
    chains=[quiz_chain,review_chain],input_variables=['text','number','subject','tone','response_json'],
    output_variables=['quiz','review'],verbose=True
)

In [20]:
file_path=r"C:\Users\HP\OneDrive\Desktop\MCQGEN\data.txt"
file_path

'C:\\Users\\HP\\OneDrive\\Desktop\\MCQGEN\\data.txt'

In [22]:
with open(file_path, 'r', encoding='utf-8') as file:
    Text=file.read()


In [ ]:
print(Text)

In [25]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct option": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct option": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct option": "correct answer"}}'

In [70]:
with get_openai_callback() as cb:
    response=generats_evalution_chain(
        {
            'text':Text,
            'number':5,
            'subject':'machine learnig',
            'tone':'simple',
            'response_json':RESPONSE_JSON
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

        Text:Machine learning (ML) is a branch of artificial intelligence (AI) focused on enabling computers and machines to imitate the way that humans learn, to perform tasks autonomously, and to improve their performance and accuracy through experience and exposure to more data.

UC Berkeley breaks out the learning system of a machine learning algorithm into three main parts.

A Decision Process: In general, machine learning algorithms are used to make a prediction or classification. Based on some input data, which can be labeled or unlabeled, your algorithm will produce an estimate about a pattern in the data.

An Error Function: An error function evaluates the prediction of the model. If there are known examples, an error function can make a comparison to assess the accuracy of the model.

A Model Optimization Process: If the model can fit better to the data points in the trainin

In [71]:
print(f'total token:{cb.total_tokens}')
print(f'prompt token:{cb.prompt_tokens}')
print(f'completion token:{cb.completion_tokens}')
print(f'total cost:{cb.total_cost}')

total token:5171
prompt token:4598
completion token:573
total cost:0.008043


In [72]:
print(response.get('quiz'))

{
    '1': {
        'mcq': 'What is the main focus of machine learning?',
        'options': {
            'a': 'Imitating human learning',
            'b': 'Performing tasks autonomously',
            'c': 'Improving performance through data exposure',
            'd': 'All of the above'
        },
        'correct option': 'd'
    },
    '2': {
        'mcq': 'Which type of learning uses labeled datasets?',
        'options': {
            'a': 'Supervised learning',
            'b': 'Unsupervised learning',
            'c': 'Semi-supervised learning',
            'd': 'Reinforcement learning'
        },
        'correct option': 'a'
    },
    '3': {
        'mcq': 'What is the main difference between deep learning and machine learning?',
        'options': {
            'a': 'Deep learning requires labeled datasets',
            'b': 'Machine learning is a sub-field of deep learning',
            'c': 'Deep learning can automatically determine features',
            'd': 'Machine 

In [73]:
quiz=response.get('quiz')

In [75]:
import ast

quiz_dict = ast.literal_eval(quiz)


In [82]:
quiz_table_data=[]
for key,value in quiz_dict.items():
    mcq=value['mcq']
    options=" | ".join(
        [
            f'{option}:{option_value}'
            for option, option_value in value['options'].items()
        ]
    )
    correct=value['correct option']
    quiz_table_data.append({'mcq':mcq,"choice":options,"correct":correct})

In [83]:
quiz_table_data

[{'mcq': 'What is the main focus of machine learning?',
  'choice': 'a:Imitating human learning | b:Performing tasks autonomously | c:Improving performance through data exposure | d:All of the above',
  'correct': 'd'},
 {'mcq': 'Which type of learning uses labeled datasets?',
  'choice': 'a:Supervised learning | b:Unsupervised learning | c:Semi-supervised learning | d:Reinforcement learning',
  'correct': 'a'},
 {'mcq': 'What is the main difference between deep learning and machine learning?',
  'choice': 'a:Deep learning requires labeled datasets | b:Machine learning is a sub-field of deep learning | c:Deep learning can automatically determine features | d:Machine learning is more dependent on human intervention',
  'correct': 'c'},
 {'mcq': 'Which algorithm is used for predicting numerical values based on a linear relationship?',
  'choice': 'a:Neural networks | b:Logistic regression | c:Linear regression | d:Decision trees',
  'correct': 'c'},
 {'mcq': 'What is the main challenge r

In [84]:
import pandas as pd
quiz=pd.DataFrame(quiz_table_data)

In [85]:
quiz

,mcq,choice,correct
0,What is the main focus of machine learning?,a:Imitating human learning | b:Performing task...,d
1,Which type of learning uses labeled datasets?,a:Supervised learning | b:Unsupervised learnin...,a
2,What is the main difference between deep learn...,a:Deep learning requires labeled datasets | b:...,c
3,Which algorithm is used for predicting numeric...,a:Neural networks | b:Logistic regression | c:...,c
4,What is the main challenge related to implemen...,a:Technological singularity | b:AI impact on j...,c


In [86]:
quiz.to_csv("machine learning.csv",index=False)